# Variational AutoEncoders

With `Variational AutoEncoders` we change the bottleneck of the DeepAutoEncoder in a way that it takes two outputs from the encoder (take two values from encoder and calculates mean and standard deviation from them):
   - mean
   - standar deviation

Then the bottleneck calculates z of these values and passes it to the decoder.
$$z = \mu + e^{0.5\sigma} * \epsilon  $$
$\mu$ = mean, $\sigma$ = standard deviation, $\epsilon$ = random sample

Decoder then decodes the modified compressed version of image so the output is an image that the model did not see in the training datatset.
![alt text](VariationalAutoEncoder.png "Title")

#### Data

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

C:\Users\pk764\miniconda3\envs\data\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def map_image(image, label):
    image = tf.cast(image, dtype=tf.float32)
    image = image / 255.0
    image = tf.reshape(image, shape=(28, 28, 1,))
    return image

In [4]:
BATCH_SIZE=128
LATENT_DIM=2

In [5]:
train_dataset=tfds.load('mnist', as_supervised=True, split="train")
train_dataset=train_dataset.map(map_image)
train_dataset=train_dataset.shuffle(1024).batch(BATCH_SIZE)

val_dataset=tfds.load('mnist', as_supervised=True, split="test")
val_dataset=val_dataset.map(map_image)
val_dataset=val_dataset.batch(BATCH_SIZE)

#### Model

`Sampling class`
$$z = \mu + e^{0.5\sigma} * \epsilon  $$

$\mu$ = mean, $\sigma$ = standard deviation, $\epsilon$ = random sample

In [42]:
class Sampling(tf.keras.layers.Layer):
    def call(self,inputs):
        mu,sigma=inputs
        
        batch=tf.shape(mu)[0]
        dim=tf.shape(mu)[1]
        epsilon=tf.keras.backend.random_normal(shape=(batch,dim))
        
        return mu+epsilon*tf.exp(0.5*sigma)

`encoder`

In [43]:
def encoder(inputs,latent_dim):
    conv_1=tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
    bnorm_1=tf.keras.layers.BatchNormalization()(conv_1)
    conv_2=tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(bnorm_1)
    bnorm_2=tf.keras.layers.BatchNormalization()(conv_2)
    
    flat=tf.keras.layers.Flatten()(bnorm_2)
    
    dense=tf.keras.layers.Dense(20,activation='relu')(flat)
    bnorm_3=tf.keras.layers.BatchNormalization()(dense)
    
    mu=tf.keras.layers.Dense(latent_dim)(bnorm_3)
    sigma=tf.keras.layers.Dense(latent_dim)(bnorm_3)
    
    return mu,sigma,bnorm_2.shape

In [44]:
def encoder_model(latent_dim,input_shape):
    inputs=tf.keras.layers.Input(shape=(input_shape))
    mu,sigma,conv_shape=encoder(inputs,latent_dim)
    z=Sampling()((mu,sigma))
    model=tf.keras.models.Model(inputs,outputs=[mu,sigma,z])
    return model,conv_shape

encoder model ouputs z for decoder, but also mmu and sigma. They are needed for the `kl_loss function` and back propagation.

`decoder`

In [45]:
def decoder(inputs,conv_shape):
    units=conv_shape[1]*conv_shape[2]*conv_shape[3]
    x=tf.keras.layers.Dense(units,activation='relu')(inputs)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Reshape((conv_shape[1],conv_shape[2],conv_shape[3]))(x)
    x=tf.keras.layers.Conv2DTranspose(64,(3,3),padding='same',activation='relu')(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Conv2DTranspose(32,(3,3),padding='same',activation='relu')(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Conv2DTranspose(1,(3,3),strides=1,padding='same',activation='sigmoid')(x)
    
    return x

In [46]:
def decoder_model(latent_dim,conv_shape):
    inputs=tf.keras.layers.Input(shape=(latent_dim,))
    outputs=decoder(inputs,conv_shape)
    model=tf.keras.models.Model(inputs,outputs)
    return model

`loss`

In [47]:
def kl_reconstruction_loss(mu,sigma):
    kl_loss=1+sigma-tf.square(mu)-tf.math.exp(sigma)
    return -0.5*tf.reduce_mean(kl_loss)

`model`

In [48]:
def model(encoder,decoder,input_shape):
    inputs=tf.keras.layers.Input(shape=(input_shape))
    z, mu, sigma = encoder(inputs)
    outputs=decoder(z)
    model=tf.keras.models.Model(inputs=inputs,outputs=outputs)
    loss=kl_reconstruction_loss(mu,sigma)
    model.add_loss(loss)
    return model

#### Training

In [49]:
encoder, conv_shape = encoder_model(latent_dim=LATENT_DIM, input_shape=(28,28,1,))
decoder = decoder_model(latent_dim=LATENT_DIM, conv_shape=conv_shape)
vae = model(encoder, decoder, input_shape=(28,28,1,))

In [50]:
optimizer=tf.keras.optimizers.Adam()
loss_metric=tf.keras.metrics.Mean()
bce_loss=tf.keras.losses.BinaryCrossentropy()

In [51]:
random_vector_for_generation=tf.random.normal(shape=[16, LATENT_DIM])
epochs=100

In [ ]:
for epoch in range(epochs):
    for step, x_batch_train in enmumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed=vae(x_batch_train)
            flatted_inputs=tf.reshape(x_batch_train,shape=[-1])
            flatted_outputs=tf.reshape(reconstructed,shape=[-1])
            loss=bce_loss(flatted_inputs,flatted_outputs)*784
            loss+=sum(vae.losses)
        grads=tape.gradient(loss,vae.trainable_weights)
        optimizer.apply_gradients(zip(grads,vae.trainable_weights))
        loss_metric(loss)